# Get Coast to Coast Assists and Buckets using PBP data


In [14]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/frustration_coast"

league = "nba"
season_yr = "2023"
season_type = "Regular Season"
games_id = pbp_season(league=league,season_yr=season_yr ,season_type=season_type)
games_list = pbp_games(games_id)

Number of games:  91


  0%|          | 0/91 [00:00<?, ?it/s]

100%|██████████| 91/91 [00:01<00:00, 58.20it/s]

Number of bad games:  0


In [37]:
year = int(season_yr)
player_dict = get_players(league = 'NBA', from_year = year, to_year = year)
team_dict = teams.get_teams() # Creating Team Dictionary
team_id = [x['id'] for x in team_dict ]

## Coast to Coast Assists

In [38]:
from pbpstats.resources.enhanced_pbp import FieldGoal
from pbpstats.resources.enhanced_pbp import Rebound
pos_store = []
Rebound_pID = []
Assist_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 5:
                if possession_event.previous_event.player1_id == possession_event.player2_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        Assist_pID.append(possession_event.player2_id)
                        print ("Assist Player: {0}".format(possession_event.player2_id))
                        ii += 1
                        # print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

100%|██████████| 91/91 [00:00<00:00, 328.11it/s]


In [41]:
Player_Name = []
a = Assist_pID
wrong_id = []
Assist_pID1, Assists = np.unique(a, return_counts=True)  
for pID in Assist_pID1:
    abc = False
    for player in player_dict:
        if player['pID'] == pID:
            abc= True
            Player_Name.append(player['Name'])
    if not abc:
        wrong_id.append(pID)
print(len(Assist_pID1))
print(len(Player_Name))
print(len(wrong_id))

145
145
0


In [48]:
player_dict

[{'pID': 1630173, 'Name': 'Precious Achiuwa', 'From': 2020, 'To': 2023},
 {'pID': 1628389, 'Name': 'Bam Adebayo', 'From': 2017, 'To': 2023},
 {'pID': 1630534, 'Name': 'Ochai Agbaji', 'From': 2022, 'To': 2023},
 {'pID': 1630583, 'Name': 'Santi Aldama', 'From': 2021, 'To': 2023},
 {'pID': 1629638,
  'Name': 'Nickeil Alexander-Walker',
  'From': 2019,
  'To': 2023},
 {'pID': 1628960, 'Name': 'Grayson Allen', 'From': 2018, 'To': 2023},
 {'pID': 1628386, 'Name': 'Jarrett Allen', 'From': 2017, 'To': 2023},
 {'pID': 203937, 'Name': 'Kyle Anderson', 'From': 2014, 'To': 2023},
 {'pID': 203507, 'Name': 'Giannis Antetokounmpo', 'From': 2013, 'To': 2023},
 {'pID': 203648, 'Name': 'Thanasis Antetokounmpo', 'From': 2015, 'To': 2023},
 {'pID': 1630175, 'Name': 'Cole Anthony', 'From': 2020, 'To': 2023},
 {'pID': 1628384, 'Name': 'OG Anunoby', 'From': 2017, 'To': 2023},
 {'pID': 1627853, 'Name': 'Ryan Arcidiacono', 'From': 2017, 'To': 2023},
 {'pID': 1630166, 'Name': 'Deni Avdija', 'From': 2020, 'To': 

In [42]:
# Player_Name = []
# a = Assist_pID
# Assist_pID1, Assists = np.unique(a, return_counts=True)  
# for pID in Assist_pID1:
#     Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
# Player_Name = list(itertools.chain(*Player_Name))

In [43]:
len(Player_Name)

145

In [44]:
data = pd.DataFrame({'Assist_pID':Assist_pID1,'Player_Name':Player_Name,'Assists':Assists})
data = data.sort_values(by=['Assists'],ascending=False)
data = data.reset_index(drop=True)
data.head()

,Assist_pID,Player_Name,Assists
0,2544,LeBron James,6
1,203999,Nikola Jokic,6
2,1630567,Scottie Barnes,5
3,1629029,Luka Doncic,5
4,1627732,Ben Simmons,5


## Coast to Coast Buckets

In [45]:
pos_store = []
Rebound_pID = []
FGM_pID = []
i = 0
ii = 0
for game in tqdm(games_list):
    for possession in game.possessions.items:
        for possession_event in possession.events:
            if (isinstance(possession_event, FieldGoal) and possession_event.is_made and not possession_event.is_assisted) and (isinstance(possession_event.previous_event, Rebound)) and possession_event.seconds_since_previous_event <= 8:
                if possession_event.previous_event.player1_id == possession_event.player1_id:
                    if not possession_event.previous_event.oreb :
                        pos_store.append(possession)
                        Rebound_pID.append(possession_event.previous_event.player1_id)
                        print ("Rebound Player: {0}".format(possession_event.previous_event.player1_id))
                        FGM_pID.append(possession_event.player1_id)
                        print ("FGM Player: {0}".format(possession_event.player1_id))
                        ii += 1
                        print("ii = {0}".format(ii))
    i += 1
    # print("i = {0}".format(i))
    clear_output(wait=True)

100%|██████████| 91/91 [00:00<00:00, 376.04it/s]


In [46]:
Player_Name = []
a = FGM_pID
FGM_pID1, FGM = np.unique(a, return_counts=True)  
for pID in FGM_pID1:
    Player_Name.append([player['Name'] for player in player_dict if player['pID'] == pID]) 
Player_Name = list(itertools.chain(*Player_Name))

In [47]:
data = pd.DataFrame({'FGM_pID':FGM_pID1,'Player_Name':Player_Name,'FGM':FGM})
data = data.sort_values(by=['FGM'],ascending=False)
data = data.reset_index(drop=True)
data.head()

,FGM_pID,Player_Name,FGM
0,1628983,Shai Gilgeous-Alexander,7
1,1627759,Jaylen Brown,6
2,203897,Zach LaVine,6
3,2544,LeBron James,5
4,202699,Tobias Harris,5
